## LFR benchmark for Altmap vs Map Eq
### Compare altmap to map eq using networkx

In [2]:
from altmap.altmap_helpers.lfr import *

# show plots in separate window
%pylab

# init rc params
init_plt_params()
fig_dir = '../../../../'

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [14]:
bench_type_str = '2400n'
# bench_type_str = '25mu'
acc_results = BenchmarkResults.read_csv(f'./lfr_results/acc_' + bench_type_str + '.csv')
infomap_results = BenchmarkResults.read_csv(f'./lfr_results/infomap_' + bench_type_str + '.csv')
altmap_results = BenchmarkResults.read_csv(f'./lfr_results/altmap_' + bench_type_str + '.csv')
altmap_sci_results = BenchmarkResults.read_csv(f'./lfr_results/altmap_sci_' + bench_type_str + '.csv')
sci_results = BenchmarkResults.read_csv(f'./lfr_results/sci_' + bench_type_str + '.csv')

print (f'Minimum number of realizations is {np.min(infomap_results.actual_realizations)}.')
print (f'Actual realizations are {infomap_results.actual_realizations}.')
print (f'Average realizations are {np.mean(infomap_results.actual_realizations)}.')


# LS linear fit to acc data
A = np.ones((acc_results.num_datapoints, 2))
A[:,0] = acc_results.var_list
y = acc_results.mean_scores.reshape((acc_results.num_datapoints, 1))
theta = np.linalg.pinv(A).dot(y)

# compute linear approximation
mu_list = np.linspace(0.15, 0.75, 25)
acc_approx = mu_list * theta[0] + theta[1]
mu_cp = (0.26 - theta[1]) / theta[0] # compute 'critical point' of acc where ami performance starts dropping
print (f'Critical point is at mu = {mu_cp[0]}.')


Minimum number of realizations is 77.
Actual realizations are [88 90 82 88 86 82 87 77 89 87 86 84 87 88 86 81 86 91 86 86 84 87 83 89
 87].
Average realizations are 85.88.
Critical point is at mu = 0.30061583846775763.


In [18]:
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))
# plt.title(f'LFR benchmark, N = {N} nodes')
# ax.plot([0.5, 0.5], [0, 1], color='k')
ax.plot([mu_cp, mu_cp], [0,1], color='k', zorder = 11)

ln1 = plot_benchmark_results(infomap_results, color='royalblue', marker='x', label='Infomap', lower_bound=0.0, 
                       upper_bound=1.0)
# plot_benchmark_results(sci_results, color='crimson', marker='s', label='SCI')
ln2 = plot_benchmark_results(altmap_results, color='darkorange', marker='^', label='Synthesizing Infomap')
ln3 = plot_benchmark_results(altmap_sci_results, color='seagreen', marker='o', label='Synthesizing Infomap with SCI')

ax_acc = ax.twinx()
ax.patch.set_alpha(0)
ax.set_zorder(10)
ax_acc.set_zorder(1)
ax_acc.set_ylim([0, 0.35])
ax_acc.plot(acc_results.var_list, acc_results.mean_scores, 's', color='crimson')
ax_acc.plot(mu_list, acc_approx, '--', color='crimson', linewidth=2)
ln4 = [plt.Line2D([0], [0], linestyle='--', marker='s', color='crimson', lw=2, label='ACC with linear approximation')]

ax.grid('both')
ax.set_xlabel('Mixing parameter $\mu$')
ax.set_ylabel(r'$\mathrm{AMI}(\mathcal{Y},\mathcal{Y}_{true})$')
ax_acc.set_ylabel(r'$\mathrm{ACC}(\mathcal{Y}_{true})$', color='crimson')
ax_acc.tick_params(axis='y', labelcolor='crimson')
leg = ln1 + ln2 + ln3 + ln4
labels = [l.get_label() for l in leg]
ax.legend(leg, labels, loc=1)
plt.tight_layout()

In [19]:
fig_path = fig_dir + 'lfr_' + bench_type_str + '_ami.pdf'
plt.savefig(fig_path, dpi=1200, format='pdf')


In [15]:
min_err = np.min(np.min([infomap_results.mean_errors - infomap_results.std_errors,\
                         altmap_results.mean_errors - altmap_results.std_errors,\
                         altmap_sci_results.mean_errors - altmap_sci_results.std_errors]))

max_err = np.max(np.max([infomap_results.mean_errors + infomap_results.std_errors,\
                         altmap_results.mean_errors + altmap_results.std_errors,\
                         altmap_sci_results.mean_errors + altmap_sci_results.std_errors]))
                                                  
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))
# plt.title(f'LFR benchmark, N = {N} nodes')
# plt.plot([0.5, 0.5], [min_err-0.2, max_err+0.2], 'r')
ax.plot([mu_cp, mu_cp], [min_err-0.2, max_err+0.2], color='k')

plot_benchmark_results(infomap_results, type='errors', color='royalblue', marker='x', label='Infomap', lower_bound=-1.0)
# plot_benchmark_results(sci_results, type='errors', color='crimson', marker='s', label='SCI')
plot_benchmark_results(altmap_results, type='errors', color='darkorange', marker='^', label='Synthesizing Infomap')
plot_benchmark_results(altmap_sci_results, type='errors', color='seagreen', marker='o', label='Synthesizing Infomap '
                                                                                              'with SCI')
plt.grid()
plt.xlabel('Mixing parameter $\mu$')
plt.ylabel(r'Mean relative error $\bar{e}_\theta$')
plt.legend(loc='upper left')
plt.tight_layout()

In [16]:
fig_path = fig_dir + 'lfr_' + bench_type_str + '_rendc.pdf'
plt.savefig(fig_path, dpi=600, format='pdf')



In [112]:
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))

ln1=plot_benchmark_results(infomap_results, color='royalblue', marker='x', label='Infomap', lower_bound=0.0, 
                       upper_bound=1.0)
ln2=plot_benchmark_results(altmap_results, color='darkorange', marker='^', label='Synthesizing Infomap')
ln3=plot_benchmark_results(altmap_sci_results, color='seagreen', marker='o', label='Synthesizing Infomap with SCI')

ax_acc = ax.twinx()
ax.patch.set_alpha(0)
ax.set_zorder(10)
ax_acc.set_zorder(1)
ln4=plot_benchmark_results(acc_results, color='crimson', marker='s', label='Average Community Clustering', 
                           plot_uncertainty=False)

altmap_span = np.max(altmap_results.mean_scores) - np.min(altmap_results.mean_scores)
acc_span = np.max(acc_results.mean_scores) - np.min(acc_results.mean_scores)
acc_ax_span = acc_span / altmap_span
ax_lower = np.min(acc_results.mean_scores) - acc_ax_span / 2
ax_upper = np.max(acc_results.mean_scores) + acc_ax_span / 2
# ax_acc.set_ylim([ax_lower, ax_upper])

ax.grid('both')
ax.set_xlabel('Number of nodes $N$')
ax.set_ylabel(r'$\mathrm{AMI}(\mathcal{Y},\mathcal{Y}_{true})$')
ax_acc.set_ylabel(r'$\mathrm{ACC}(\mathcal{Y}_{true})$', color='crimson')
ax_acc.tick_params(axis='y', labelcolor='crimson')
leg = ln1 + ln2 + ln3 + ln4
labs = [l.get_label() for l in leg]
ax.legend(leg, labs, loc='best')
plt.tight_layout()

In [53]:
fig_path = fig_dir + 'lfr_' + bench_type_str + '_ami.pdf'
plt.savefig(fig_path, dpi=1200, format='pdf')